In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from collections import namedtuple
from scipy.stats import linregress
import os

In [2]:
data = ['BC 438.9 94.9 5.65 0.11',
'CO 258.3 55.9 3.31 0.07',
'BP 189.1 40.9 4.49 0.08',
'BS 96.0 20.8 8.55 0.15',
'TCV 565.9 122.4 4.83 0.09',
'IN 517.1 111.8 4.27 0.07',
'GC 541.4 117.1 4.88 0.09',
'CE 328.6 71.1 9.71 0.01',
'ODY 22.3 4.8 6.59 0.07',
'MA 1513.8 327.4 9.85 0.18',
'GR 222.7 48.2 2.97 0.01',
'ME 853.3 184.5 3.61 0.05',
'CM 370.7 80.2 3.57 0.06',
'AA 337.1 72.9 8.25 0.00',
'BI 113.2 24.5 4.21 0.07',
'EL 396.0 85.6 3.66 0.10',
'PO 25.8 5.6 4.93 0.10',
'CRPP 440.3 95.2 5.25 0.12',
'MX 775.2 167.6 4.84 0.15',
'BM 660.0 142.7 7.05 0.11',
'CH 1291.2 279.2 7.11 0.15',
'DIA 0.0 0.0 7.69 0.15',
'PH 1213.4 262.4 6.34 0.21',
'AI 1066.8 230.7 11.79 0.22']

In [50]:
data2 = ['BC 2 17480 418,491 1,603,596',
'CO 2 11901 477,008 943,653',
'BP 2 10442 457,861 691,031',
'BS 2 10267 509,183 350,860',
'TCV 2 6095 318,209 2,067,675',
'IN 2 24073 1,260,041 1,889,430',
'GC 1 26586 1,465,755 1,978,120',
'CE 1 16655 1,003,313 1,200,598',
'ODY 2 4092 253,199 81,410',
'MA 1 14018 889,271 5,531,370',
'GR 1 9997 649,081 813,804',
'ME 2 17151 1,126,830 3,118,001',
'CM 1 18663 1,251,411 1,354,652',
'AA + SG 2 18389 1,306,603 1,231,934',
'BI 1 4496 345,679 413,651',
'EL 2 22127 1,728,630 1,447,090',
'PO 2 692 64,607 94,326',
'CRPP 2 10831 928,960 1,608,750',
'MX 2 25868 2,600,901 2,832,408',
'BM 2 19697 2,121,607 2,411,721',
'CH + STT 1 28,986 3,217,870 4,717,985',
'DIA 2 847 105,136 -',
'PH 1 23581 3,036,870 4,433,829',
'AI 2 17674 2,768,898 3,898,106']

In [51]:
# Initialize arrays
Buildings = []
E_A       = []
E_B       = []
kth       = []
ksun      = []
temp_type = []
Ath       = []

## TODO:
#### Convert units of kth
#### 

In [52]:
for i in data:
    lst = i.split()
    Buildings.append(lst[0])
    E_A.append(lst[1])
    E_B.append(lst[2])
    kth.append(lst[3])
    ksun.append(lst[4])
for i in data2:
    lst = i.split()
    Ath.append(lst[2]) # m2
    temp_type.append(lst[1])


In [62]:
# Q NEEDS TO BE CHANGED
Q = 1.2
for i in range(len(Buildings)):
    filename = 'building_dat/moes' + Buildings[i] + '.dat'
    writeToText(filename, Buildings[i], temp_type[i], Ath[i], kth[i], ksun[i], Q, E_A[i], E_B[i])

In [63]:
def writeToText (path, buildingname, tempflag, floorarea, kth, ksun, Q, elec1, elec2): 
    f = open (path, "w+")
    line = 'let Buildings := Buildings union {' + buildingname + '};\n'
    f.write(line)
    if tempflag:
        line = 'let MediumTempBuildings := MediumTempBuildings union {' + buildingname + '};\n'
    else:
        line = 'let LowTempBuildings := LowTempBuildings union {' + buildingname + '};\n'
    f.write(line)
    f.write('###########BUILDING SPECIFIC PARAMETER  Time  Independend ##########################\n')
    line = 'let FloorArea[' + buildingname + '] := ' + floorarea + ';\n'
    f.write(line)
    line = 'let k_th[' + buildingname + ']:= ' + kth + ';\n'
    f.write(line)
    line = 'let k_sun[' + buildingname + ']:= ' + ksun + ';\n'
    f.write(line)
    line = 'let specQ_people[' + buildingname + ']:= ' + str(Q) + ';\n'
    f.write(line)
    f.write('###########BUILDING SPECIFIC PARAMETER  Time  Dependend ##########################\n')
    line = 'let specElec[' + buildingname + ',1] := ' + elec1 + ';\n'
    f.write(line)
    line = 'let specElec[' + buildingname + ',2] := ' + elec2 + ';\n'
    f.close()

In [ ]:


let Buildings := Buildings union {'Building1'};								# Add "Building1" to set of Buildings
let MediumTempBuildings := MediumTempBuildings union {'Building1'}; 		# Decide if it belongs to Medium or Low Temperature Buildings

###########BUILDING SPECIFIC PARAMETER  Time  Independend ##########################

let FloorArea['Building1'] := 20000;
let k_th['Building1']:= 0.003 ;												# ATTENTION unit in kW
let k_sun['Building1']:= 0.01;
let specQ_people['Building1']:= 0.0025;										# ATTENTION unit in kW

###########BUILDING SPECIFIC PARAMETER  Time  Dependend ##########################
let specElec['Building1',1] := 0.0035;										# ATTENTION unit in kW
let specElec['Building1',2] := 0.0030;										# ATTENTION unit in kW